<a href="https://colab.research.google.com/github/Svein-Tore/colab/blob/main/pi_regulator_niva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# Regulering av tankanlegg – SIMC og Ziegler–Nichols
#
# Denne notebooken brukes til å:
#  - modellere et tankanlegg som et FOPDT-system
#  - sammenligne SIMC- og ZN-innstilling av PI-regulator
#  - se hvordan valg av λ påvirker responsen
#  - koble simulering mot virkelig anlegg
#
# Endre parametere med sliderne og observer hva som skjer.
# ============================================================
import importlib
import subprocess
import sys

def ensure_package(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", pkg]
        )

# Sørg for nødvendige pakker
ensure_package("numpy")
ensure_package("matplotlib")
ensure_package("control")
ensure_package("ipywidgets")
# importer pakker
import control as ctrl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, FloatSlider, ToggleButton
from google.colab import files

# -----------------------------
# Last opp fil
# -----------------------------
uploaded = files.upload()
if uploaded:
  filename = list(uploaded.keys())[0]
  # Pandas – automatisk separator, desimal=komma
  df = pd.read_csv(filename, sep=None, engine='python', decimal=',')
  tid_data = df.iloc[:,0].values
  niva_data = df.iloc[:,1].values
# =========================
# PI-regulator: SIMC eller ZN
# =========================
def pi_controller(Kp_process, T, L, lam, ZN=False):
    s = ctrl.TransferFunction.s

    if ZN:
        # Ziegler–Nichols PI (åpen sløyfe / reaksjonskurve)
        Kc = 0.9 * T / (Kp_process * L)
        Ti = 3 * L
        tuning = "Ziegler–Nichols"
    else:
        # SIMC PI
        Kc = T / (Kp_process * (lam + L))
        Ti = T
        tuning = "SIMC"

    C = Kc * (1 + 1/(Ti*s))
    return C, Kc, Ti, tuning


# =========================
# FOPDT-prosess
# =========================
def fopdt_system(K, T, L):
    num_pade, den_pade = ctrl.pade(L, 1)
    dead_time = ctrl.TransferFunction(num_pade, den_pade)
    G_no_delay = ctrl.TransferFunction([K], [T, 1])
    return G_no_delay * dead_time


# =========================
# Plot-funksjon med y0
# =========================
def plot_step_response(K=1.0, T=97.0, L=6.0, A=20.0, y0=5.0, lam=48.5, ZN=False):

    G = fopdt_system(K, T, L)
    C, Kc, Ti, tuning = pi_controller(K, T, L, lam, ZN)

    # Lukket sløyfe
    T_closed = ctrl.feedback(C * G, 1)

    # Simuleringstid
    t_end =max(5*(lam + L), 5*T)
    t = np.linspace(0, t_end, 1500)

    # Sprang endring i set punkt lik A-y0
    t, y = ctrl.step_response(T_closed *(A-y0), t)
    y = y0 + y  # forskyv kurven med y0

    plt.figure(figsize=(10, 5))
    plt.plot(t, y)
    plt.title(f"Sprangrespons – {tuning} PI")
    plt.xlabel("Tid (s)")
    plt.ylabel("y(t)")
    plt.grid(True)
    if uploaded:
      plt.plot(tid_data, niva_data, "b.", markersize=3, label="Måledata")

    # Vis regulatorparametere
    plt.text(0.7,0.2 ,
        f"Tuning: {tuning}\nKp = {Kc:.3f}\nTi = {Ti:.2f} s\nStartnivå y0 = {y0:.2f}",
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.85)
    )

    plt.show()


# =========================
# Interaktive kontroller
# =========================
interact(
    plot_step_response,
    K=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='K'),
    T=FloatSlider(value=97.0, min=1.0, max=200.0, step=1.0, description='T'),
    L=FloatSlider(value=6.0, min=0.1, max=30.0, step=0.1, description='L'),
    A=FloatSlider(value=80.0, min=0.1, max=100.0, step=0.1, description='set-punkt'),
    y0=FloatSlider(value=5.0, min=0.0, max=100.0, step=0.1, description='y0'),
    lam=FloatSlider(value=97*0.5, min=0.3*97, max=2*97, step=1.0, description='λ'),
    ZN=ToggleButton(value=False, description='Endre tuning')
)